In [1]:
#!/usr/bin/env python
# coding: utf-8

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2,application_1691346772328_0003,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
import os
import sys
import pytz
import numpy as np
import datetime
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import udf, lpad, translate
from datetime import datetime 
from datetime import timedelta
from datetime import date
from dateutil.relativedelta import relativedelta
from pyspark.sql.types import *
from pyspark.sql.functions import count, avg

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
agora=datetime.now(pytz.timezone('America/Sao_Paulo'))
dthproc=agora.strftime("%Y%m%d%H%M%S")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
spark = SparkSession.builder.appName('T_bi_politica_credito_imob_modelo_001').getOrCreate()
sqlContext=SQLContext(spark.sparkContext)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
_DIA_ = '20230804'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
file='bureau_20230805.csv'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
ts_file_generation = file.split("_")[-1].replace('.csv','') + '00'
print(ts_file_generation)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2023080500

In [8]:
bucket_raw = 'bkt-datalake-grupo-01-hackathon-podacademy/{dia}/002_raw'.format(dia=_DIA_)
bucket_ingestion = 'bkt-datalake-grupo-01-hackathon-podacademy/{dia}/001_ingestion'.format(dia=_DIA_)
bucket_control = 'bkt-datalake-grupo-01-hackathon-podacademy/{dia}/005_control'.format(dia=_DIA_)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
output_lake = 't_bureau_20230805'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
full_path_ingestion = 's3://{bkt}/{file}'.format(bkt=bucket_ingestion,file=file)
print(full_path_ingestion)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://bkt-datalake-grupo-01-hackathon-podacademy/20230804/001_ingestion/bureau_20230805.csv

In [11]:
pouso = spark.read.format(
       "com.databricks.spark.csv").option(
       "header", "true").option(
       "encoding", "ISO-8859-1").option(
       "encoding", "UTF-8").option(
       "inferSchema", "false").option(
        "delimiter", ',').load(
       full_path_ingestion)
pouso.registerTempTable("pouso")
pouso.cache()
qtd=pouso.count()
print('records ingestion data:', qtd)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

records ingestion data: 1716428

In [12]:
for col in pouso.columns:
    print(col + ',')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SK_ID_CURR,
SK_ID_BUREAU,
CREDIT_ACTIVE,
CREDIT_CURRENCY,
DAYS_CREDIT,
CREDIT_DAY_OVERDUE,
DAYS_CREDIT_ENDDATE,
DAYS_ENDDATE_FACT,
AMT_CREDIT_MAX_OVERDUE,
CNT_CREDIT_PROLONG,
AMT_CREDIT_SUM,
AMT_CREDIT_SUM_DEBT,
AMT_CREDIT_SUM_LIMIT,
AMT_CREDIT_SUM_OVERDUE,
CREDIT_TYPE,
DAYS_CREDIT_UPDATE,
AMT_ANNUITY,

In [13]:
lake = spark.sql(     
    """
        select
            -- padrao para todas as cargas no datalake--
            
            int(date_format(current_date,'yyyyMM')) as ref,
            int(date_format(current_date,'yyyyMM')) as ref_partition,
            
            {tsfileger} as ts_file_generation,
            {tsfileger} as ts_file_generation_partition,
            
            {pdthproc} as ts_proc,
            {pdthproc} as ts_proc_partition,
        
            -- campos do arquivo --

            SK_ID_CURR,
            SK_ID_BUREAU,
            CREDIT_ACTIVE,
            CREDIT_CURRENCY,
            DAYS_CREDIT,
            CREDIT_DAY_OVERDUE,
            DAYS_CREDIT_ENDDATE,
            DAYS_ENDDATE_FACT,
            AMT_CREDIT_MAX_OVERDUE,
            CNT_CREDIT_PROLONG,
            AMT_CREDIT_SUM,
            AMT_CREDIT_SUM_DEBT,
            AMT_CREDIT_SUM_LIMIT,
            AMT_CREDIT_SUM_OVERDUE,
            CREDIT_TYPE,
            DAYS_CREDIT_UPDATE,
            AMT_ANNUITY
            
        from
            pouso
    """.format(tsfileger=ts_file_generation,pdthproc=dthproc))
lake.registerTempTable("lake")
lake.cache()
lake.count()  

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1716428

In [14]:
lake.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- ref: integer (nullable = true)
 |-- ref_partition: integer (nullable = true)
 |-- ts_file_generation: integer (nullable = false)
 |-- ts_file_generation_partition: integer (nullable = false)
 |-- ts_proc: long (nullable = false)
 |-- ts_proc_partition: long (nullable = false)
 |-- SK_ID_CURR: string (nullable = true)
 |-- SK_ID_BUREAU: string (nullable = true)
 |-- CREDIT_ACTIVE: string (nullable = true)
 |-- CREDIT_CURRENCY: string (nullable = true)
 |-- DAYS_CREDIT: string (nullable = true)
 |-- CREDIT_DAY_OVERDUE: string (nullable = true)
 |-- DAYS_CREDIT_ENDDATE: string (nullable = true)
 |-- DAYS_ENDDATE_FACT: string (nullable = true)
 |-- AMT_CREDIT_MAX_OVERDUE: string (nullable = true)
 |-- CNT_CREDIT_PROLONG: string (nullable = true)
 |-- AMT_CREDIT_SUM: string (nullable = true)
 |-- AMT_CREDIT_SUM_DEBT: string (nullable = true)
 |-- AMT_CREDIT_SUM_LIMIT: string (nullable = true)
 |-- AMT_CREDIT_SUM_OVERDUE: string (nullable = true)
 |-- CREDIT_TYPE: string (nullable 

In [15]:
lake.show(4,truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------------+------------------+----------------------------+--------------+-----------------+----------+------------+-------------+---------------+-----------+------------------+-------------------+-----------------+----------------------+------------------+--------------+-------------------+--------------------+----------------------+---------------+------------------+-----------+
|ref   |ref_partition|ts_file_generation|ts_file_generation_partition|ts_proc       |ts_proc_partition|SK_ID_CURR|SK_ID_BUREAU|CREDIT_ACTIVE|CREDIT_CURRENCY|DAYS_CREDIT|CREDIT_DAY_OVERDUE|DAYS_CREDIT_ENDDATE|DAYS_ENDDATE_FACT|AMT_CREDIT_MAX_OVERDUE|CNT_CREDIT_PROLONG|AMT_CREDIT_SUM|AMT_CREDIT_SUM_DEBT|AMT_CREDIT_SUM_LIMIT|AMT_CREDIT_SUM_OVERDUE|CREDIT_TYPE    |DAYS_CREDIT_UPDATE|AMT_ANNUITY|
+------+-------------+------------------+----------------------------+--------------+-----------------+----------+------------+-------------+---------------+-----------+------------------+-------------------+--

In [16]:
# +------+-------------+------------------+----------------------------+--------------+-----------------+----------+------------+-------------+---------------+-----------+------------------+-------------------+-----------------+----------------------+------------------+--------------+-------------------+--------------------+----------------------+---------------+------------------+-----------+
# |ref   |ref_partition|ts_file_generation|ts_file_generation_partition|ts_proc       |ts_proc_partition|SK_ID_CURR|SK_ID_BUREAU|CREDIT_ACTIVE|CREDIT_CURRENCY|DAYS_CREDIT|CREDIT_DAY_OVERDUE|DAYS_CREDIT_ENDDATE|DAYS_ENDDATE_FACT|AMT_CREDIT_MAX_OVERDUE|CNT_CREDIT_PROLONG|AMT_CREDIT_SUM|AMT_CREDIT_SUM_DEBT|AMT_CREDIT_SUM_LIMIT|AMT_CREDIT_SUM_OVERDUE|CREDIT_TYPE    |DAYS_CREDIT_UPDATE|AMT_ANNUITY|
# +------+-------------+------------------+----------------------------+--------------+-----------------+----------+------------+-------------+---------------+-----------+------------------+-------------------+-----------------+----------------------+------------------+--------------+-------------------+--------------------+----------------------+---------------+------------------+-----------+
# |202308|202308       |2023080500        |2023080500                  |20230805113813|20230805113813   |215354    |5714462     |Closed       |currency 1     |-497       |0                 |-153.0             |-153.0           |null                  |0                 |91323.0       |0.0                |null                |0.0                   |Consumer credit|-131              |null       |
# |202308|202308       |2023080500        |2023080500                  |20230805113813|20230805113813   |215354    |5714463     |Active       |currency 1     |-208       |0                 |1075.0             |null             |null                  |0                 |225000.0      |171342.0           |null                |0.0                   |Credit card    |-20               |null       |
# |202308|202308       |2023080500        |2023080500                  |20230805113813|20230805113813   |215354    |5714464     |Active       |currency 1     |-203       |0                 |528.0              |null             |null                  |0                 |464323.5      |null               |null                |0.0                   |Consumer credit|-16               |null       |
# |202308|202308       |2023080500        |2023080500                  |20230805113813|20230805113813   |215354    |5714465     |Active       |currency 1     |-203       |0                 |null               |null             |null                  |0                 |90000.0       |null               |null                |0.0                   |Credit card    |-16               |null       |
# +------+-------------+------------------+----------------------------+--------------+-----------------+----------+------------+-------------+---------------+-----------+------------------+-------------------+-----------------+----------------------+------------------+--------------+-------------------+--------------------+----------------------+---------------+------------------+-----------+
# only showing top 4 rows

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
path_raw = os.path.join('s3://',bucket_raw,output_lake)
print(path_raw)
lake.coalesce(1).write. \
partitionBy("ref_partition","ts_file_generation_partition","ts_proc_partition"). \
parquet(path_raw,mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://bkt-datalake-grupo-01-hackathon-podacademy/20230804/002_raw/t_bureau_20230805

In [18]:
varDataFile = output_lake

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
controle = spark.sql(    
        """
            select
                '{tb}' as name_file,
                '{tb}' as name_file_partition,
                
                ref,
                ref_partition,
                
                ts_file_generation,
                ts_file_generation_partition,
                
                ts_proc,
                ts_proc_partition,
                
                count(*) as qtd_registros
            from 
                lake as a
            group by
                1,2,3,4,5,6,7,8
            order by
                1,2,3,4,5,6,7,8
        """.format(tb=varDataFile))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
controle.registerTempTable("controle")
controle.cache()
qtd=controle.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
controle.show(qtd,truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+-------------------+------+-------------+------------------+----------------------------+--------------+-----------------+-------------+
|name_file        |name_file_partition|ref   |ref_partition|ts_file_generation|ts_file_generation_partition|ts_proc       |ts_proc_partition|qtd_registros|
+-----------------+-------------------+------+-------------+------------------+----------------------------+--------------+-----------------+-------------+
|t_bureau_20230805|t_bureau_20230805  |202308|202308       |2023080500        |2023080500                  |20230806155119|20230806155119   |1716428      |
+-----------------+-------------------+------+-------------+------------------+----------------------------+--------------+-----------------+-------------+

In [22]:
# +-----------------+-------------------+------+-------------+------------------+----------------------------+--------------+-----------------+-------------+
# |name_file        |name_file_partition|ref   |ref_partition|ts_file_generation|ts_file_generation_partition|ts_proc       |ts_proc_partition|qtd_registros|
# +-----------------+-------------------+------+-------------+------------------+----------------------------+--------------+-----------------+-------------+
# |t_bureau_20230805|t_bureau_20230805  |202308|202308       |2023080500        |2023080500                  |20230806155119|20230806155119   |1716428      |
# +-----------------+-------------------+------+-------------+------------------+----------------------------+--------------+-----------------+-------------+

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
path_control = os.path.join('s3://',bucket_control,'tb_0001_controle_procesamento_raw')
print(path_control)
controle.coalesce(1).write.\
parquet(path_control,mode='append')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://bkt-datalake-grupo-01-hackathon-podacademy/20230804/005_control/tb_0001_controle_procesamento_raw